# Variable / Operations Identification

In [1]:



Nodes = Set()
CompGraph = [] # TODO: rename to edges
Jacobians = IdDict()


function JacobiansAdd(source, sink, Jacobian)
    merge!(Jacobians, (source, sink) => Jacobian) 
end

function GenerateID()
    ID = convert(Int64, floor(10000000 * rand()))
    while ID in Nodes
        ID = convert(Int64, floor(10000000 * rand()))
    end
    union!(Nodes, ID)
    return ID
end

mutable struct Tracked{T}
    val::T
    ID::Int64
    Tracked(val) = return new{typeof(val)}(val, GenerateID())
    Tracked(val, ID) = return new{typeof(val)}(val, ID)
end


Operations to manipulate the directed acyclic computationnal graphs created using Julia's built-in dictionnary data structure.

In [2]:

function AddEdge(Edges, source, sink)
    if source in Edges merge!(Edges, source=>Set([sink, get(Edges, source)...]))
    else merge!(Edges, source=>Set(sink)) end 
end

function RemoveEdge(Edges, source, sink)
    if source in keys(Edges) delete!(get(Edges, source, Nothing), sink) end
end

function ReverseEdges(Edges)::IdDict
    ReversedEdges = IdDict{}
    for (source, sinks) in edges
        for sink in sinks AddEdge(ReversedEdges, source, sink) end
    end
    return ReversedEdges
end

function HasIncomingEdge(Edges, Node)::Bool
    for sinks in values(Edges)  if (Node in sinks) return true end end
    return false
end


function KahnTopoSort(Nodes::Set, Edges::IdDict)
    # Kahn's topological sorting algorithm
    # edges is a DAG in the form of a dictionnary (source => sinks)
    Sorted = []
    @show NoIncomingEdges = Set( [Node for Node in Nodes  if !HasIncomingEdge(Edges, Node)] )
    while !isempty(NoIncomingEdges)
        source = pop!(NoIncomingEdges)
        Sorted = cat(Sorted, [source], dims=1)
        sinks = get(Edges, source, false)
        if sinks!=false for sink in sinks
            RemoveEdge(Edges, source, sink)
            if !HasIncomingEdge(Edges, sink) NoIncomingEdges = union(NoIncomingEdges, Set(sink)) end
        end end
    end
    return Sorted
end


Nodes = Set([1, 2, 3, 4, 5])
Edges = IdDict(1=>Set([2, 3]), 4=>Set([5]), 2=>Set([4, 5]), 3=>Set([4]))
@show KahnTopoSort(Nodes, Edges) #should return [5, 4, 2, 3, 1] or [5, 2, 4, 3, 1]


NoIncomingEdges = Set([Node for Node = Nodes if !(HasIncomingEdge(Edges, Node))]) = Set([1])
KahnTopoSort(Nodes, Edges) = 

Any[1, 2, 3, 4, 5]


5-element Vector{Any}:
 1
 2
 3
 4
 5

# Wengert List
Every function in our program, no matter the complexity, can be represented as a wengert list of very basic operations. A basic expression in a Wengert List follows:
maximum of two operands, and an operator.

We need our program to create a trace (which can be described as a wengert list) by going into a forward pass with note taking operators. Note taking operators are modified basic operators that write down in a global dictionnary the basic operands and the **symbolic identifiers** of their arguments (identif. from the caller, not the func. def).

Here, the constructer "Tracked" is introduced. If a particular tensor is tracked, it means that every atomic computation involving it needs to be considered as a function which needs to be differentiated with respect to it. The output tensor becomes tracked, since there is a possibility it will be used to compute the loss, y, in the end.

Here the dictionnary is problematic, since only single links can be store. 

In [3]:
import Base.:+
import Base.:-
import Base.:*
import Base.:/
import Base.:^


GetJacobian(f::typeof(+), a::Real, b::Tracked) = 1
GetJacobian(f::typeof(+), a::Tracked, b::Real) = 1
GetJacobian(f::typeof(-), a::Real, b::Tracked) = -1
GetJacobian(f::typeof(-), a::Tracked, b::Real) = 1
GetJacobian(f::typeof(*), a::Real, b::Tracked) = a
GetJacobian(f::typeof(*), a::Tracked, b::Real) = b
GetJacobian(f::typeof(/), a::Real, b::Tracked) = (-1)/b.val^2
GetJacobian(f::typeof(/), a::Tracked, b::Real) = 1/b



for op in (Symbol(+), Symbol(-), Symbol(*), Symbol(/), Symbol(^))

    eval(:(global function ($op)(a::Real, b::Tracked)
    ID = GenerateID()
    J = GetJacobian(($op), a, b)
    CompGraphAdd(b.ID, ID)
    JacobiansAdd(b.ID, ID, J)
    return Tracker(($op)(a.val, b.val), ID) end))
    

    eval(:(global function ($op)(a::Tracked, b::Real)
    ID = GenerateID()
    J = GetJacobian(($op), a, b)
    CompGraphAdd(a.ID, ID)
    JacobiansAdd(a.ID, ID, J)
    return Tracker(($op)(a.val, b.val), ID) end))

    eval(:(global function ($op)(a::Tracked, b::Tracked)
    ID = GenerateID()
    Ja = GetJacobian(($op), a, b.val)
    CompGraphAdd(a.ID, ID)
    JacobiansAdd(a.ID, ID, Ja)
    Jb = GetJacobian(($op), a.val, b)
    CompGraphAdd(b.ID, ID)
    JacobiansAdd(b.ID, ID, Jb)
    return Tracker(($op)(a.val, b.val), ID) end))

end



In [4]:

function Backprop(ID)
    Parents = CompGraph.get(ID)
    for p in Parents
        Jacobians.set(p, Jacobians.get(p) * Jacobians.get(p=>f))
        Backprop(f)
    end
end

function Backprop(x, w, f)

    y = f(w, x) # forward pass (intermediate jacobians are computed)
    Backprop(y.ID) # backwards pass (application of the chain rule)
    TopoSortNodes = KahnTopoSort(Nodes, Edges)
    for Node in 
    for p in Parents
        Jacobians.set(p, Jacobians.get(p) * Jacobians.get(p=>f))
        Backprop(f)
    end
end

BackpropStart (generic function with 1 method)

In [5]:
# every program can be deconstructed as a wengert list

oepli(w, x) = 2 * (((w + x) - w) * w) + w^2

w = @show Tracked(7)
x = 3
#println(CompGraph)
#println(Jacobians)
@show BackpropStart(x, w, oepli)

UndefVarError: UndefVarError: `IdSpace` not defined

# Autodiff Matrix
The function which will be differentiated is of the form $y = f(\theta_1, \theta_2, \dots)$, where the $\theta$'s are the Trackedeters to be updated. The input is considered a constant in the function. 

$$f(\theta_1, \theta_2, \dots) = f(\theta_1, f(\theta_2))$$

$$\frac{d f(g(x))}{dx} = \frac{d f}{d g} \frac{dg}{dx}$$




In [6]:

function G(W1, W2, W3, x)
    return sum(sin.(W3*(cos.(W2*(sin.(W1*x))))))
end

function *(A::Param, B::Matrix)
    merge!(CompGraph, IdDict{Any, Any}((A.ID)=>(previous)))
    Jacobian = B
    merge!(Jacobians, IdDict{Any, Any}((A.ID)=>(Jacobian)))
    global previous = A.ID
    return (*)(A.ID, B) 
end

y = G(Param(rand(5,5)), Param(rand(5,5)), Param(rand(5,5)), rand(5,5))
@show Jacobians
@show CompGraph

UndefVarError: UndefVarError: `Param` not defined